In [106]:
#Funcion para lectura de datos .gpx
import gpxpy
import pandas as pd
import sqlite3
from geopy.distance import distance

def read_gpx(file: str) -> pd.DataFrame:
    df = None
    points = []
    with open(file) as f:
        gpx = gpxpy.parse(f)

    for segment in gpx.tracks[0].segments:
        for p in segment.points:
            points.append({
                'time':p.time,
                'latitude':p.latitude,
                'longitude':p.longitude,
                'elevation':p.elevation
            })
    df = pd.DataFrame.from_records(points)
    
    return df

In [107]:
#Datos
df1 = read_gpx('recovery.01-Mar-2022-1533.gpx')
df2 = read_gpx('recovery.05-Mar-2022.1025.gpx')
df3 = read_gpx('recovery.25-May-2022-0907.gpx')

In [108]:
#Copias a los datos originales
Data1 = df1.copy()
Data2 = df2.copy()
Data3 = df3.copy()

In [109]:
Data1.isnull().any()
Data1.duplicated().any()
# Ahora se mira si existen filas duplicadas
duplicados = Data2.duplicated()
duplicados
filas_duplicadas = Data1[duplicados]
filas_duplicadas
Data1[Data1.duplicated('time')]

C:\Users\CCOSORIO\AppData\Local\Temp\ipykernel_23960\2707494956.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filas_duplicadas = Data1[duplicados]


,time,latitude,longitude,elevation


In [110]:
#Separar Campos de fecha y hora
def separar_campos_fechas(df):

    df["Date_Time"] = pd.to_datetime(df["time"], format="%y-%m-%d").dt.tz_localize(None)
    df['date'] = pd.to_datetime(df['Date_Time'].dt.date)
    df['time'] = df['Date_Time'].dt.time
    df['year'] = df['Date_Time'].dt.year
    df['month'] = df['Date_Time'].dt.month
    df['day'] = df['Date_Time'].dt.day
    df['hour'] = df['Date_Time'].dt.hour
    df['minutes'] = df['Date_Time'].dt.minute
    df['seconds'] = df['Date_Time'].dt.second
     
    return df

In [111]:
separar_campos_fechas(Data2)

,time,latitude,longitude,elevation,Date_Time,date,year,month,day,hour,minutes,seconds
0,00:04:34,6.180893,-75.577296,1589.639492,2022-03-04 00:04:34,2022-03-04,2022,3,4,0,4,34
1,00:04:35,6.180912,-75.577295,1588.165859,2022-03-04 00:04:35,2022-03-04,2022,3,4,0,4,35
2,00:04:43,6.180921,-75.577312,1591.435879,2022-03-04 00:04:43,2022-03-04,2022,3,4,0,4,43
3,00:04:46,6.180897,-75.577270,1587.116055,2022-03-04 00:04:46,2022-03-04,2022,3,4,0,4,46
4,00:04:48,6.180874,-75.577276,1586.772305,2022-03-04 00:04:48,2022-03-04,2022,3,4,0,4,48
...,...,...,...,...,...,...,...,...,...,...,...,...
18588,15:25:40,6.255059,-75.590578,1481.257603,2022-03-05 15:25:40,2022-03-05,2022,3,5,15,25,40
18589,15:25:42,6.255073,-75.590597,1480.845982,2022-03-05 15:25:42,2022-03-05,2022,3,5,15,25,42
18590,15:25:44,6.255086,-75.590614,1480.847569,2022-03-05 15:25:44,2022-03-05,2022,3,5,15,25,44
18591,15:25:46,6.255103,-75.590619,1480.186069,2022-03-05 15:25:46,2022-03-05,2022,3,5,15,25,46


In [112]:
#Calcular la distancia entre puntos (Formula de Haversine)

def distancia_puntos(row):
    Cordenada1 = (row['latitude'], row['longitude'])
    Cordenada2 = (Data2.iloc[0]['latitude'], Data2.iloc[0]['longitude'])
    Distancia = distance(Cordenada1, Cordenada2).meters
    return Distancia

In [113]:
Data2['distance'] = Data2.apply(distancia_puntos, axis=1)

In [114]:
Data2.head(50)

,time,latitude,longitude,elevation,Date_Time,date,year,month,day,hour,minutes,seconds,distance
0,00:04:34,6.180893,-75.577296,1589.639492,2022-03-04 00:04:34,2022-03-04,2022,3,4,0,4,34,0.000000
1,00:04:35,6.180912,-75.577295,1588.165859,2022-03-04 00:04:35,2022-03-04,2022,3,4,0,4,35,2.160142
2,00:04:43,6.180921,-75.577312,1591.435879,2022-03-04 00:04:43,2022-03-04,2022,3,4,0,4,43,3.611928
3,00:04:46,6.180897,-75.577270,1587.116055,2022-03-04 00:04:46,2022-03-04,2022,3,4,0,4,46,2.945579
4,00:04:48,6.180874,-75.577276,1586.772305,2022-03-04 00:04:48,2022-03-04,2022,3,4,0,4,48,3.069086
5,00:04:49,6.180849,-75.577266,1586.357265,2022-03-04 00:04:49,2022-03-04,2022,3,4,0,4,49,5.914801
6,00:04:50,6.180862,-75.577277,1587.002285,2022-03-04 00:04:50,2022-03-04,2022,3,4,0,4,50,4.008764
7,00:04:51,6.180877,-75.577289,1586.430508,2022-03-04 00:04:51,2022-03-04,2022,3,4,0,4,51,1.886874
8,00:05:21,6.180953,-75.577350,1584.878506,2022-03-04 00:05:21,2022-03-04,2022,3,4,0,5,21,8.913317
9,00:05:23,6.181004,-75.577227,1582.133633,2022-03-04 00:05:23,2022-03-04,2022,3,4,0,5,23,14.556799


In [115]:
#Calcular la velocidad 
def velocidad(df, columna):
    df['delta'] = df[columna].diff().fillna(pd.Timedelta(seconds=0))
    df['delta'] = df['delta'].dt.seconds
    df['speed'] = df['distance'] / df['delta']
    return df

In [116]:
velocidad(Data2, 'Date_Time').head(50)

,time,latitude,longitude,elevation,Date_Time,date,year,month,day,hour,minutes,seconds,distance,delta,speed
0,00:04:34,6.180893,-75.577296,1589.639492,2022-03-04 00:04:34,2022-03-04,2022,3,4,0,4,34,0.000000,0,NaN
1,00:04:35,6.180912,-75.577295,1588.165859,2022-03-04 00:04:35,2022-03-04,2022,3,4,0,4,35,2.160142,1,2.160142
2,00:04:43,6.180921,-75.577312,1591.435879,2022-03-04 00:04:43,2022-03-04,2022,3,4,0,4,43,3.611928,8,0.451491
3,00:04:46,6.180897,-75.577270,1587.116055,2022-03-04 00:04:46,2022-03-04,2022,3,4,0,4,46,2.945579,3,0.981860
4,00:04:48,6.180874,-75.577276,1586.772305,2022-03-04 00:04:48,2022-03-04,2022,3,4,0,4,48,3.069086,2,1.534543
5,00:04:49,6.180849,-75.577266,1586.357265,2022-03-04 00:04:49,2022-03-04,2022,3,4,0,4,49,5.914801,1,5.914801
6,00:04:50,6.180862,-75.577277,1587.002285,2022-03-04 00:04:50,2022-03-04,2022,3,4,0,4,50,4.008764,1,4.008764
7,00:04:51,6.180877,-75.577289,1586.430508,2022-03-04 00:04:51,2022-03-04,2022,3,4,0,4,51,1.886874,1,1.886874
8,00:05:21,6.180953,-75.577350,1584.878506,2022-03-04 00:05:21,2022-03-04,2022,3,4,0,5,21,8.913317,30,0.297111
9,00:05:23,6.181004,-75.577227,1582.133633,2022-03-04 00:05:23,2022-03-04,2022,3,4,0,5,23,14.556799,2,7.278399


In [117]:
#Cargar datos a sqlite
def ingestar_sqlite(df, BD = 'Base.db', Tabla = 'Cordenadas'):
    conn = sqlite3.connect(BD)
    df.to_sql(Tabla, conn, if_exists='replace', index = False)
    df_consulta = pd.read_sql(f'SELECT * FROM {Tabla}', conn)
    conn.close()
    return df_consulta

In [118]:
ingestar_sqlite(Data2)

,time,latitude,longitude,elevation,Date_Time,date,year,month,day,hour,minutes,seconds,distance,delta,speed
0,00:04:34.000000,6.180893,-75.577296,1589.639492,2022-03-04 00:04:34,2022-03-04 00:00:00,2022,3,4,0,4,34,0.000000,0,NaN
1,00:04:35.000000,6.180912,-75.577295,1588.165859,2022-03-04 00:04:35,2022-03-04 00:00:00,2022,3,4,0,4,35,2.160142,1,2.160142
2,00:04:43.000000,6.180921,-75.577312,1591.435879,2022-03-04 00:04:43,2022-03-04 00:00:00,2022,3,4,0,4,43,3.611928,8,0.451491
3,00:04:46.000000,6.180897,-75.577270,1587.116055,2022-03-04 00:04:46,2022-03-04 00:00:00,2022,3,4,0,4,46,2.945579,3,0.981860
4,00:04:48.000000,6.180874,-75.577276,1586.772305,2022-03-04 00:04:48,2022-03-04 00:00:00,2022,3,4,0,4,48,3.069086,2,1.534543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18588,15:25:40.000000,6.255059,-75.590578,1481.257603,2022-03-05 15:25:40,2022-03-05 00:00:00,2022,3,5,15,25,40,8332.580245,2,4166.290123
18589,15:25:42.000000,6.255073,-75.590597,1480.845982,2022-03-05 15:25:42,2022-03-05 00:00:00,2022,3,5,15,25,42,8334.369610,2,4167.184805
18590,15:25:44.000000,6.255086,-75.590614,1480.847569,2022-03-05 15:25:44,2022-03-05 00:00:00,2022,3,5,15,25,44,8336.119769,2,4168.059884
18591,15:25:46.000000,6.255103,-75.590619,1480.186069,2022-03-05 15:25:46,2022-03-05 00:00:00,2022,3,5,15,25,46,8338.184960,2,4169.092480
